# Electrical network

## Introduction

Wait a few seconds until **Python interaction ready!** is shown in the top.

**After that follow the instructions in the Grasple Exercise.**

On this webpage the system of differential equations

$$
\begin{cases} \dfrac{dI_1}{dt}=-\left(\dfrac{R_1+R_2}{L}\right)I_1+\dfrac{R_2}{L}I_2,\\\\ \dfrac{dI_2}{dt}= -\left(\dfrac{R_1+R_2}{L}\right)I_1+\left(\dfrac{R_2}{L}-\dfrac{1}{R_2C}\right)I_2,\end{cases}
$$

is considered, with initial conditions

$$
\left\{\begin{array}{rcl}I_1(0) &=&1,\\I_2(0)&=&0.\end{array}\right.$$


This webpage solves this initial-value problem symbolically, using <code>sympy</code>, and then uses <code>numpy</code> and <code>matplotlib</code> to display the obtained *exact* solution.

In [ ]:
import micropip
await micropip.install("ipympl")
# %matplotlib widget
%matplotlib ipympl
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
    
def Electrical_network(R1,R2,L,C):
    
    def solve_ivp(R1,R2,L,C):
        I10 = 1
        I20 = 0
        a = -(R1+R2)/L
        b = R2/L
        c = ( R2/L - 1/R2/C )
        t = sp.symbols('t')
        I1 = sp.Function('I1')
        I2 = sp.Function('I2')
        ode = [ sp.diff(I1(t),t) - ( a*I1(t) + b*I2(t) ) , sp.diff(I2(t),t) - (a*I1(t) + c*I2(t) ) ]
        sol = sp.dsolve(ode)
        sol_new = sol.copy()
        for index,s in enumerate(sol):
            s = sp.N(s)
            # find weird sin's, cos's and exp's and make them contain only floats
            sins = s.find(sp.sin)
            for x in sins:
                arg_x = x.args[0]
                s = s.replace(arg_x,sp.N(arg_x))
            coss = s.find(sp.cos)
            for x in coss:
                arg_x = x.args[0]
                s = s.replace(arg_x,sp.N(arg_x))
            exps = s.find(sp.exp)
            for x in exps:
                arg_x = x.args[0]
                s = s.replace(arg_x,sp.N(arg_x))
            sol_new[index] = s

        constants = sp.solve([sol_new[0].rhs.subs(t,0)-I10,sol_new[1].rhs.subs(t,0)-I20],sp.symbols('C1 C2'))
        I1 = sol_new[0].rhs.subs(constants)
        I2 = sol_new[1].rhs.subs(constants)
        return I1,I2

    t = sp.symbols('t')

    I1,I2 = solve_ivp(R1,R2,L,C)

    lam_I1 = sp.lambdify(t,I1,modules=['numpy'])
    lam_I2 = sp.lambdify(t,I2,modules=['numpy'])
    t_end = 40+C
    # search for adequate t_end
    I1_end = lam_I1(t_end)
    I2_end = lam_I2(t_end)
    norm_end = np.sqrt(I1_end**2+I2_end**2)
    while norm_end>1e-16:
        t_end = t_end*10
        I1_end = lam_I1(t_end)
        I2_end = lam_I2(t_end)
        norm_end = np.sqrt(I1_end**2+I2_end**2)
    t_vals = np.logspace(-16, np.log10(t_end), 100001)
    t_vals[0] = 0
    I1_vals = lam_I1(t_vals)
    I2_vals = lam_I2(t_vals)

    maxI1 = np.max(np.abs(I1_vals))
    maxI2 = np.max(np.abs(I2_vals))
    maxI = np.max([maxI1,maxI2])

    plt.close('all')
    fig = plt.figure()
    plt.plot(t_vals,I1_vals,label='$I_1$')
    plt.plot(t_vals,I2_vals,label='$I_2$')
    plt.xlim(0,40+C)
    plt.grid()
    plt.xlabel('$t$')
    plt.ylabel('$Current$')
    plt.legend()
    # fig.canvas.toolbar_visible = Treue

    fig2 = plt.figure()
    plt.plot(I1_vals,I2_vals)
    plt.xlabel('$I_1(t)$')
    plt.ylabel('$I_2(t)$')
    plt.axis('equal')
    plt.grid();
    fig2.canvas.toolbar_visible = False
    
    pass

## INPUT CELL

In the cell below you should put the values of the parameters you see in *Grasple*.

*Please note*: Each attempt in Grasple contains different values for the parameters.

After you run the cell the results of the calculations are shown with the set parameter values.

If you point your mouse below a figure, controls will appear that you can use to pan and zoom.

In [ ]:
''' Set the given values in the following lines '''
R_1 = 0.8
R_2 = 2.22
L = 1
C = 3

''' The next line performs the calculations and shows the results. '''
Electrical_network(R_1,R_2,L,C)